# EDA
The goal of this notebook is to:
- remove null values
- transform data into a usable state for modelling
- perform data analysis to uncover patterns
- eliminate any sources of data leakage

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
df = pd.read_csv("./data/pokedex_raw.csv", index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1028 entries, 0 to 1027
Data columns (total 50 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   pokedex_number    1028 non-null   int64  
 1   name              1028 non-null   object 
 2   german_name       938 non-null    object 
 3   japanese_name     938 non-null    object 
 4   generation        1028 non-null   int64  
 5   status            1028 non-null   object 
 6   species           1028 non-null   object 
 7   type_number       1028 non-null   int64  
 8   type_1            1028 non-null   object 
 9   type_2            542 non-null    object 
 10  height_m          1028 non-null   float64
 11  weight_kg         1027 non-null   float64
 12  abilities_number  1028 non-null   int64  
 13  ability_1         1025 non-null   object 
 14  ability_2         513 non-null    object 
 15  ability_hidden    810 non-null    object 
 16  total_points      1028 non-null   float64


### Dropping Unecessary Columns
German/Japanese names have no use for us. Let's drop them.

In [3]:
df.drop(["german_name", "japanese_name"], axis=1, inplace=True)

## Dealing with Nulls
base_friendship, base_experience, catch_rate, and percentage_male have the same number of null values. Let's investigate

In [4]:
df.loc[df["base_friendship"].isnull() & df["base_experience"].isnull() & df["catch_rate"].isnull() & df["percentage_male"].isnull(), ["name", "generation", "status", "base_friendship", "base_experience", "catch_rate", "percentage_male"]]

,name,generation,status,base_friendship,base_experience,catch_rate,percentage_male
69,Galarian Meowth,1,Normal,NaN,NaN,NaN,NaN
100,Galarian Ponyta,1,Normal,NaN,NaN,NaN,NaN
102,Galarian Rapidash,1,Normal,NaN,NaN,NaN,NaN
109,Galarian Farfetch'd,1,Normal,NaN,NaN,NaN,NaN
142,Galarian Weezing,1,Normal,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1023,Zacian Hero of Many Battles,8,Legendary,NaN,NaN,NaN,NaN
1024,Zamazenta Crowned Shield,8,Legendary,NaN,NaN,NaN,NaN
1025,Zamazenta Hero of Many Battles,8,Legendary,NaN,NaN,NaN,NaN
1026,Eternatus,8,Legendary,NaN,NaN,NaN,NaN


A lot of the nulls seem to be either of type "Galarian", or from generation 8. We can also see that the four fields are null in the same rows

In [5]:
df.loc[df["base_friendship"].isnull() & (df["generation"] != 8)].loc[:, ["name", "pokedex_number", "generation", "status"]]

,name,pokedex_number,generation,status
69,Galarian Meowth,52,1,Normal
100,Galarian Ponyta,77,1,Normal
102,Galarian Rapidash,78,1,Normal
109,Galarian Farfetch'd,83,1,Normal
142,Galarian Weezing,110,1,Normal
156,Galarian Mr. Mime,122,1,Normal
267,Galarian Corsola,222,2,Normal
314,Galarian Zigzagoon,263,3,Normal
316,Galarian Linoone,264,3,Normal
648,Galarian Darumaka,554,5,Normal


In [7]:
df = df[~df["name"].str.contains("Galarian")]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1014 entries, 0 to 1027
Data columns (total 48 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   pokedex_number    1014 non-null   int64  
 1   name              1014 non-null   object 
 2   generation        1014 non-null   int64  
 3   status            1014 non-null   object 
 4   species           1014 non-null   object 
 5   type_number       1014 non-null   int64  
 6   type_1            1014 non-null   object 
 7   type_2            534 non-null    object 
 8   height_m          1014 non-null   float64
 9   weight_kg         1013 non-null   float64
 10  abilities_number  1014 non-null   int64  
 11  ability_1         1011 non-null   object 
 12  ability_2         506 non-null    object 
 13  ability_hidden    798 non-null    object 
 14  total_points      1014 non-null   float64
 15  hp                1014 non-null   float64
 16  attack            1014 non-null   float64


In [8]:
df[df["generation"] == 8].isnull().sum()

pokedex_number       0
name                 0
generation           0
status               0
species              0
type_number          0
type_1               0
type_2              48
height_m             0
weight_kg            1
abilities_number     0
ability_1            1
ability_2           42
ability_hidden      13
total_points         0
hp                   0
attack               0
defense              0
sp_attack            0
sp_defense           0
speed                0
catch_rate          88
base_friendship     88
base_experience     88
growth_rate          0
egg_type_number      0
egg_type_1           0
egg_type_2          47
percentage_male     88
egg_cycles           0
against_normal       0
against_fire         0
against_water        0
against_electric     0
against_grass        0
against_ice          0
against_fight        0
against_poison       0
against_ground       0
against_flying       0
against_psychic      0
against_bug          0
against_rock         0
against_gho

### Filling Gen 8 Nulls
We need percentage_male, catch_rate, base_friendship, base_experience. We use webscraping with BeautifulSoup and bulbapedia.bulbagarden.net

In [9]:
import requests
from bs4 import BeautifulSoup
import re

In [10]:
soup = BeautifulSoup(requests.get("https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_effort_value_yield").content, "html.parser")
expTable = soup.find_all("tr")
def getBaseExp(pNum, tablerow):
    for row in tablerow[1:]:
        if int(re.search("[0-9]+", str(row.findChildren("td")[0])).group()) == pNum:
            return int(re.search("[0-9]+", str(row.findChildren("td")[3])).group())
for pNum in df.loc[df["generation"] == 8, "pokedex_number"]:
    df.loc[df["pokedex_number"] == pNum, "base_experience"] = getBaseExp(pNum, expTable)

In [11]:
soup = BeautifulSoup(requests.get("https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_base_friendship").content, "html.parser")
friendTable = soup.find_all("tr")
def getBaseFriendship(pNum, tablerow):
    for row in tablerow[2:]:
        if int(re.search("[0-9]+", str(row.findChildren("td")[0])).group()) == pNum:
            return int(re.search("[0-9]+", str(row.findChildren("td")[4])).group())
for pNum in df.loc[df["generation"] == 8, "pokedex_number"]:
    df.loc[df["pokedex_number"] == pNum, "base_friendship"] = getBaseFriendship(pNum, friendTable)

In [12]:
soup = BeautifulSoup(requests.get("https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_catch_rate").content, "html.parser")
catchTable = soup.find_all("tr")
def getCatchRate(pNum, tablerow):
    for row in tablerow[2:]:
        if int(re.search("[0-9]+", str(row.findChildren("th")[0])).group()) == pNum:
            return int(re.search("[0-9]+", str(row.findChildren("td")[2])).group())
for pNum in df.loc[df["generation"] == 8, "pokedex_number"]:
    df.loc[df["pokedex_number"] == pNum, "catch_rate"] = getCatchRate(pNum, catchTable)

In [13]:
soup = BeautifulSoup(requests.get("https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_gender_ratio").content, "html.parser")
genderTable = soup.find_all("table", {"class": "sortable"})
def getPercentageMale(pNum, table):
    for i in range(len(table)):
        rows = table[i].findChildren("tr")[1:]
        for row in rows:
            if int(re.search("[0-9]+", str(row.findChildren("td")[0])).group()) == pNum:
                if 0 <= i < 2: return 100.
                elif 2 <= i < 4: return 87.5
                elif 4 <= i < 6: return 75.
                elif 6 <= i < 8: return 50.
                elif 8 <= i < 10: return 25.
                elif 10 <= i < 11: return 12.5
                elif 11 <= i < 13: return 0.
                else: return np.nan
for pNum in df.loc[df["generation"] == 8, "pokedex_number"]:
    df.loc[df["pokedex_number"] == pNum, "percentage_male"] = getPercentageMale(pNum, genderTable)

## Miscellaneous Nulls

#### Meltan and Melmetal
There are still 2 null rows for the four fields above, and they correspond to meltan and melmetal (pokedex numbers 808, 809)

In [14]:
for pNum in range(808, 810):
    df.loc[df["pokedex_number"] == pNum, "percentage_male"] = getPercentageMale(pNum, genderTable)
    df.loc[df["pokedex_number"] == pNum, "catch_rate"] = getCatchRate(pNum, catchTable)
    df.loc[df["pokedex_number"] == pNum, "base_friendship"] = getBaseFriendship(pNum, friendTable)
    df.loc[df["pokedex_number"] == pNum, "base_experience"] = getBaseExp(pNum, expTable)

#### Weight
We see one missing value for weight.

In [19]:
df.loc[df["weight_kg"].isnull()]

,pokedex_number,name,generation,status,species,type_number,type_1,type_2,height_m,weight_kg,...,against_ground,against_flying,against_psychic,against_bug,against_rock,against_ghost,against_dragon,against_dark,against_steel,against_fairy
1027,890,Eternatus Eternamax,8,Legendary,Gigantic Pokémon,2,Poison,Dragon,100.0,NaN,...,2.0,1.0,2.0,0.5,1.0,1.0,2.0,1.0,1.0,1.0


A quick manual search shows that Eternatus Eternamax is a pokemon so large, it has no known weight, so we set it to int max for now. 

In [20]:
df.loc[df["weight_kg"].isnull(), "weight_kg"] = np.finfo(np.float64).max

#### Ability 1
There are 3 missing values for ability 1. 

In [15]:
df.loc[df["ability_1"].isnull(), ["name", "pokedex_number", "ability_1", "ability_2"]]

,name,pokedex_number,ability_1,ability_2
33,Partner Pikachu,25,NaN,NaN
170,Partner Eevee,133,NaN,NaN
1027,Eternatus Eternamax,890,NaN,NaN


A search for partner pokemon shows they are a special form of the original pokemon with elevated base stats, so we remove them.

In [16]:
df = df[~df["name"].str.contains("Partner")]

As for Eternatus Eternamax, we fill its null values with its base form, Eternatus

In [17]:
df[df["name"] == "Eternatus Eternamax"].isnull().any() & ~df[df["name"] == "Eternatus"].isnull().any()

pokedex_number      False
name                False
generation          False
status              False
species             False
type_number         False
type_1              False
type_2              False
height_m            False
weight_kg            True
abilities_number    False
ability_1            True
ability_2           False
ability_hidden      False
total_points        False
hp                  False
attack              False
defense             False
sp_attack           False
sp_defense          False
speed               False
catch_rate          False
base_friendship     False
base_experience     False
growth_rate         False
egg_type_number     False
egg_type_1          False
egg_type_2          False
percentage_male     False
egg_cycles          False
against_normal      False
against_fire        False
against_water       False
against_electric    False
against_grass       False
against_ice         False
against_fight       False
against_poison      False
against_grou

In [18]:
df.loc[df["name"] == "Eternatus Eternamax", "ability_1"] = df[df["name"] == "Eternatus"]["ability_1"].values